In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import gc
import math
import cupy as cp
import pandas as pd
from time import time

import warnings
warnings.filterwarnings('ignore')

parent = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(parent, "drp1_mito_tracker"))

from CZImage import CZImage
from extract import tif_to_array, czi_to_array, add_mito_data
from tracking import tracker, calc_threshold_fit
from mito_skel import video_skeleton_data, skeleton_as_dataframe # radi, radi_as_dataframe
from skimage.filters import threshold_otsu, threshold_li, threshold_isodata

In [ ]:
dir_name = "TODO"
filename_suffix = "czi"

In [ ]:
files = []
number = 0
for (dirpath, dirnames, filenames) in os.walk(dir_name):
    for s in filenames:
        if s.endswith('.' + filename_suffix):
            files.append(s)
            print(number, s, round(os.path.getsize(os.path.join(dir_name, s))/1024/1024/1024, 2), "GB")
            number += 1
    break

In [ ]:
begin = time()

for filename in files:
    gc.collect()
    filetime = time()
    
    base_filename = filename[:-4]
    
    mtdnatrac_path = os.path.join(dir_name, base_filename + "_mtdnatrac" + '.csv')
    drp1trac_path = os.path.join(dir_name, base_filename + "_drp1trac" + '.csv')
    mitoskel_path = os.path.join(dir_name, base_filename + "_mitoskel" + '.csv')
    
    # Uncomment to ignore existing DataFrames 
    if os.path.isfile(mtdnatrac_path) and os.path.isfile(drp1trac_path) and os.path.isfile(mitoskel_path):
        print("Files exists for", base_filename)
        continue
    
    print("Processing", base_filename)
    
    path = os.path.join(dir_name, base_filename + '.' + filename_suffix)

    # DRP1 Tracking
    # start = time()
    # data = czi_to_array(path, channels=['drp1'])
    # drp1 = data['drp1']
    # print("Load DRP1", time() - start)
    
    start = time()
    data = CZImage(path, channel=CZImage.DRP1)
    drp1_df = tracker(data, 
                      max_distance=15,
                      look_back_n_frames=1,
                      min_coords=80,#int(len(data) * 0.8), 
                      maxima_threshold_method=threshold_otsu)
    print("DRP1 Tracker", time() - start)
    
    # mtDNA Tracking
    # start = time()
    # data = czi_to_array(path, channels=['mtdna'])
    # mtdna = data['mtdna']
    # print("Load mtDNA", time() - start)
    
    start = time()
    # mtdna_df = tracker(cp.array(mtdna), max_distance=15, maxima_threshold_method=threshold_otsu)
    data = CZImage(path, channel=CZImage.MTDNA)
    mtdna_df = tracker(data, 
                       max_distance=15,
                       look_back_n_frames=1,
                       min_coords=80,#int(len(data) * 0.8),
                       maxima_threshold_method=threshold_otsu)
    print("mtDNA Tracker", time() - start)
    
    # Mitochondria Tracking
    # start = time()
    # data = czi_to_array(path, channels=['mito'])
    # mito = data['mito']
    # print("Load mito", time() - start)
    
    mito = CZImage(path, channel=CZImage.MITO)

    start = time()
    treshold, mean_threshold = calc_threshold_fit(mito, threshold_li)
    maxima_treshold, mean_maxima_threshold = calc_threshold_fit(mito, threshold_otsu)
    print("Thresholds", time() - start)
    
    start = time()
    # c = 64
    # count = math.ceil(len(mito)/c)
    # res = []
    # for i in range(count):
    #     res.append(skeleton_as_dataframe(video_skeleton_data(mito[i*c:min(len(mito), (i+1)*c)], treshold[i*c], maxima_treshold[i*c])))
    #     res[i].t += i*c
    # mito_df = pd.concat(res)
    # mito_df.index = range(len(mito_df))
    mito_df = skeleton_as_dataframe(video_skeleton_data(mito, treshold, maxima_treshold))
    mito_df.to_csv(mitoskel_path, index=False)
    print("Mito Skeleton", time() - start)
    
    start = time()
    drp1_df = add_mito_data(drp1_df, mito_df)
    drp1_df.to_csv(drp1trac_path, index=False)
    print("Add Mito to Drp1 Tracking", time() - start)
    
    start = time()
    mtdna_df = add_mito_data(mtdna_df, mito_df)
    mtdna_df.to_csv(mtdnatrac_path, index=False)
    print("Add Mito to mtDNA Tracking", time() - start)
    
    print("File time", time() - filetime)

print("Overall", time() - begin)